# Project 2


### Dataset 1

In [1]:
# Import libraries
import pandas as pd
import sys

In [2]:
# loading first dataset
df_1 = pd.read_csv('dataset_1.csv')
print(df_1)


   id    name  phone sex and age test number  term 1  term 2  term 3
0   1    Mike    134        m_12      test 1      76      84      87
1   2   Linda    270        f_13      test 1      88      90      73
2   3     Sam    210        m_11      test 1      78      74      80
3   4  Esther    617        f_12      test 1      68      75      74
4   5    Mary    114        f_14      test 1      65      67      64
5   1    Mike    134        m_12      test 2      85      80      90
6   2   Linda    270        f_13      test 2      87      82      94
7   3     Sam    210        m_11      test 2      80      87      80
8   4  Esther    617        f_12      test 2      70      75      78
9   5    Mary    114        f_14      test 2      68      70      63


In [3]:
# Multiple types of observational units are stored in the same table
df_students = df_1[['id', 'name', 'phone', 'sex and age']]
df_data = df_1[['id', 'test number', 'term 1', 'term 2', 'term 3']]


In [7]:
def split_sex_and_age(row):
    [sex, age] = row['sex and age'].split('_')
    sex = 'male' if sex == 'm' else 'female'
    return pd.Series([sex, age])

#  'sex and age' column needs to be split into 2, and drop the original column
df_students[['sex', 'age']] = df_students.apply(lambda row: split_sex_and_age(row), axis=1)
df_students.drop(['sex and age'], axis=1, inplace=True)

df_students

,id,name,phone,sex,age
0,1,Mike,134,male,12
1,2,Linda,270,female,13
2,3,Sam,210,male,11
3,4,Esther,617,female,12
4,5,Mary,114,female,14
5,1,Mike,134,male,12
6,2,Linda,270,female,13
7,3,Sam,210,male,11
8,4,Esther,617,female,12
9,5,Mary,114,female,14


## Data df

In [4]:
# Some column headers are values (term 1, term 2 and term 3), not variable names.
df_data_processed = pd.melt(df_data, id_vars=['id', 'test number'], var_name='term', value_name='marks')
# clean text from value
df_data_processed['term'] = df_data_processed.apply(lambda row: row['term'].split()[1], axis=1)

df_data_processed.head()

,id,test number,term,marks
0,1,test 1,1,76
1,2,test 1,1,88
2,3,test 1,1,78
3,4,test 1,1,68
4,5,test 1,1,65


In [5]:
# multiple variables in a column
def get_marks_for_tests(dataset, row):
	student_id = row['id']
	term = row['term']
	test1_observation = dataset.loc[(dataset['id'] == student_id) & (dataset['term'] == term) & (dataset['test number'] == 'test 1')].iloc[0]
	test2_observation = dataset.loc[(dataset['id'] == student_id) & (dataset['term'] == term) & (dataset['test number'] == 'test 2')].iloc[0]

	test1_marks = test1_observation['marks']
	test2_marks = test2_observation['marks']

	return pd.Series([test1_marks, test2_marks])

# marks for each term
df_data_processed[['test 1 marks', 'test 2 marks']] = df_data_processed.apply(
	lambda row: get_marks_for_tests(df_data_processed, row), axis=1)

In [6]:
df_data_processed.head()

,id,test number,term,marks,test 1 marks,test 2 marks
0,1,test 1,1,76,76,85
1,2,test 1,1,88,88,87
2,3,test 1,1,78,78,80
3,4,test 1,1,68,68,70
4,5,test 1,1,65,65,68


In [8]:
# drop columns ('marks' and 'test number')
df_data_processed.drop(['marks', 'test number'], axis=1, inplace=True)
# drop duplicates
df_data_processed.drop_duplicates(['id', 'term'], keep='first', inplace=True)
# sort by id
df_data_processed.sort_values(by=['id'], inplace=True)

df_data_processed

,id,term,test 1 marks,test 2 marks
0,1,1,76,85
10,1,2,84,80
20,1,3,87,90
1,2,1,88,87
11,2,2,90,82
21,2,3,73,94
2,3,1,78,80
12,3,2,74,87
22,3,3,80,80
3,4,1,68,70


# Analysis

### 1 .Analysis on performance by tests

In [9]:
df_data_processed[["test 1 marks", "test 2 marks"]].mean()

test 1 marks    76.200000
test 2 marks    79.266667
dtype: float64

Test 2 performed better because GPA was better

### 2. You can get an average per term and test

In [10]:
df_data_processed.groupby("term")[["test 1 marks", "test 2 marks"]].mean()

,test 1 marks,test 2 marks
term,,
1,75.0,78.0
2,78.0,78.8
3,75.6,81.0


### 3. Analysis by sex

In [11]:
# firt we must join data
df_joined = df_data_processed.join(df_students[["id", "sex"]].set_index('id'), on='id')

df_joined.groupby("sex")[["test 1 marks", "test 2 marks"]].mean()

,test 1 marks,test 2 marks
sex,,
female,73.777778,76.333333
male,79.833333,83.666667


### 4. Analysis by name


In [12]:
# firt we must join data
df_joined_name = df_data_processed.join(df_students[["id", "name"]].set_index('id'), on='id')

df_joined_name.groupby("name")[["test 1 marks", "test 2 marks"]].mean()


,test 1 marks,test 2 marks
name,,
Esther,72.333333,74.333333
Linda,83.666667,87.666667
Mary,65.333333,67.000000
Mike,82.333333,85.000000
Sam,77.333333,82.333333
